In [1]:
using Pkg
Pkg.activate("..");

In [2]:
using MLKernels
using Distributions
using Plots
using Random
using Optim
using LinearAlgebra
using NearestNeighbors
using SVDD
using OCALPlots
using JuMP, Gurobi

┌ Info: Recompiling stale cache file /home/trittenb/.julia/compiled/v1.1/OCALPlots/MW3RA.ji for OCALPlots [4194d700-ee83-11e8-2783-1f838a7c2787]
└ @ Base loading.jl:1184


In [ ]:
modulepath = abspath(joinpath(@__DIR__, "../src/KernelLearning"));
push!(LOAD_PATH, modulepath);
using KernelLearning

┌ Info: Recompiling stale cache file /home/trittenb/.julia/compiled/v1.1/KernelLearning.ji for KernelLearning [top-level]
└ @ Base loading.jl:1184


In [ ]:
@recipe function plot_svdd(m::SVDD.SVDDClassifier, labels::Vector{Symbol}; grid_resolution = 100, axis_overhang = 0.2, db_color=:black)
    grid_range, grid_data = OCALPlots.get_grid(extrema(m.data)..., grid_resolution, axis_overhang)
    grid_scores = reshape(SVDD.predict(m, grid_data), grid_resolution, grid_resolution)
    data_class = SVDD.classify.(SVDD.predict(m, m.data))
    
    title --> "Decision Boundary"
    
    @series begin
        seriestype := :contourf
        seriescolor --> :greens
        levels := range(0, maximum(grid_scores), length=10)
        grid_range, grid_range, grid_scores
    end

    colors = (inlier = :blue, outlier = :red)
    shapes = (inlier = :circle, outlier = :star8)

    markeralpha --> 0.7
    markersize --> 5

    for l in [:inlier, :outlier]
        markercolor := colors[l]
        for dc in [:inlier, :outlier]
            markershape := shapes[dc]
            @series begin
                seriestype := :scatter
                label := OCALPlots.get_legend_text(l, dc)
                OCALPlots.split_2d_array(m.data, (labels.==l) .& (data_class .== dc))
            end
       end
    end

    @series begin
        seriestype := :contour
        levels := [0]
        linewidth := 3
        seriescolor := [db_color]
        cbar:= false
        grid_range, grid_range, grid_scores
    end
end

In [ ]:
fntsm = Plots.font("sans-serif", pointsize=10)
fntlg = Plots.font("sans-serif", pointsize=10)
default(titlefont=fntlg, guidefont=fntlg, tickfont=fntsm, legendfont=fntsm)
default(markersize=8)

In [ ]:
solver = with_optimizer(Gurobi.Optimizer; OutputFlag=0, Threads=1)

In [ ]:
alignment(K1, K2) = dot(K1, K2) / sqrt(norm(K1) * norm(K2))

#### Data Generation

In [ ]:
Random.seed!(7);

In [ ]:
μ = 0.0
σ = 1.0 
n = 200
data = rand(MvNormal(2, σ), n)
data = hcat(data, [-2, 0], [.7, 1.6], [0,3])
labels = vcat(fill(:inlier, n), :inlier, :inlier, :outlier);

data = hcat(data, [1,-3], [-2, 3])
labels = vcat(labels, :outlier, :outlier);

In [ ]:
pools = fill(:U, 205)
pools[[1,5]] .= :Lin
pools[[n+4]] .= :Lout

In [ ]:
Plots.scatter(data[1, pools.==:U], data[2, pools.==:U], color=:lightgrey, label=:U)
Plots.scatter!(data[1, pools.==:Lin], data[2, pools.==:Lin], color=:blue, label=:Lin, markershape = :utriangle)
p = Plots.scatter!(data[1, pools.==:Lout], data[2, pools.==:Lout], color=:red, label=:Lout, legend=:topleft)

In [ ]:
savefig(p, "plots/sample_global.pdf")

#### Global Alignment on Sample

In [ ]:
y = [1/3, 1/3, -1]
K_opt = y*y'
samples = data

In [ ]:
function neg_alignment(gamma)
    K_empiric = kernelmatrix(Val(:col), GaussianKernel(gamma), samples[:, [1, 5, n+4]])
    -alignment(K_empiric, K_opt)    
end

pos_alignment(gamma) = -neg_alignment(gamma)

In [ ]:
res = Optim.optimize(neg_alignment, 0.1, 10.0, abs_tol=0.1)
gamma_global = res.minimizer

In [ ]:
plot(neg_alignment)

In [ ]:
kernelmatrix(Val(:col), GaussianKernel(res.minimizer), data[:, end-2:end])

In [ ]:
model_2 = VanillaSVDD(data)
init_strategy = FixedParameterInitialization(GaussianKernel(gamma_global), 0.04)
initialize!(model_2, init_strategy)
fit!(model_2, solver)

p = Plots.plot(model_2, fill(:inlier, size(data,2)), title="", color=:blues, legend=false)

In [ ]:
savefig(p, "plots/sample_global_svdd.pdf")

##### Alignment on ground truth

In [ ]:
y_gt = vcat(fill(1.0/(n+2), n+2), fill(1.0/3, 3))
K_gt = y_gt*y_gt';

In [ ]:
K_opt = K_gt;

In [ ]:
alignment(K1, K2) = dot(K1, K2) / sqrt(norm(K1) * norm(K2))

function neg_alignment(gamma)
    K_empiric = centerkernelmatrix(kernelmatrix(Val(:col), GaussianKernel(gamma), data))
    _K = centerkernelmatrix(K_opt)
    -alignment(K_empiric, _K)
end

pos_alignment(gamma) = -neg_alignment(gamma)

In [ ]:
plot(neg_alignment, 0, 100)

In [ ]:
res_pos = Optim.optimize(pos_alignment, 0.01, 1.0, abs_tol=0.05)
res_pos.minimizer

In [ ]:
res = Optim.optimize(neg_alignment, 0.01, res_pos.minimizer, abs_tol=0.05)
gamma_gt = res.minimizer

##### Alignment based on local neighborhood

In [ ]:
k=15
nnpool = NNPool(data, k=k);

KernelLearning.add_to_pools!(nnpool, 1, :inlier, k)
KernelLearning.add_to_pools!(nnpool, 5, :inlier, k)
# KernelLearning.add_to_pools!(nnpool, 94, :inlier, k)

KernelLearning.add_to_pools!(nnpool, n+4, :outlier, k)

In [ ]:
idx = collect(keys(nnpool.nn_pools[:Lin]) ∪ keys(nnpool.nn_pools[:Lout]));

In [ ]:
y = zeros(size(data, 2));

In [ ]:
y[idx] .= [get(nnpool.nn_pools[:Lin], id, 0) / (get(nnpool.nn_pools[:Lin], id, 0) + get(nnpool.nn_pools[:Lout], id, 0)) for id in idx]
y[idx] = ifelse.(y[idx] .> 0.5, 1, -1)

idx_in = findall(y .== 1)
idx_out = findall(y .== -1);

In [ ]:
K_opt = y * y';

In [ ]:
kdtree = KDTree(data; leafsize = k)
nn = knn(kdtree, data, k, true)[1]
reverse_nn = KernelLearning.reverseKNN(nn)
symmetric_nn = [n ∩ r for (n,r) in zip(nn, reverse_nn)];

In [ ]:
K_c = kernelmatrix(Val(:col), GaussianKernel(res.minimizer), data);

In [ ]:
mask = falses(size(K_c))

for (_, id) in enumerate(findall(nnpool.pools .!= :U)) #i ∈ L_in
    @show id
    if y[id] == 1
        # TODO: why don't we also use the observation itself, i.e., [1:end]?
        nn_in = nn[id][1:end] ∩ idx_in # L'_in ∩ NN_k(i)
        nn_out = nn[id][1:end] ∩ idx_out
    else
        nn_in = setdiff(nn[id][1:end], reverse_nn[id][1:end]) ∩ idx_in
        nn_out = symmetric_nn[id][1:end] ∩ idx_out
    end
        
    @show nn_in, nn_out

    if length(nn_in) > 0
        mask[id, nn_in[1:min(end, k)]] .= true
        mask[nn_in[1:min(end, k)], id] .= true
    end

    if length(nn_out) > 0
        mask[id, nn_out[1:min(end, k)]] .= true
        mask[nn_out[1:min(end, k)], id] .= true
    end
end

In [ ]:
M_in = reduce(union, [(nn[id][1:end] ∩ idx_in) ∪ (nn[id][1:end] ∩ idx_out) for id in findall(nnpool.pools .== :Lin)]);
M_out = reduce(union, [(setdiff(nn[id][1:end], reverse_nn[id][1:end]) ∩ idx_in) ∪ (symmetric_nn[id][1:end] ∩ idx_out) for id in findall(nnpool.pools .== :Lout)]);

In [ ]:
Plots.scatter(data[1, setdiff(findall(nnpool.pools .== :U), M_in ∪ M_out)], data[2, setdiff(findall(nnpool.pools .== :U), M_in ∪ M_out)], color=:lightgrey, label="U")
Plots.scatter!(data[1, setdiff(M_in, findall(nnpool.pools.!=:U))], data[2, setdiff(M_in, findall(nnpool.pools.!=:U))], color=:orange, label="M_in", markershape=:hexagon)
Plots.scatter!(data[1, setdiff(M_out, findall(nnpool.pools.!=:U))], data[2, setdiff(M_out, findall(nnpool.pools.!=:U))], color=:green, label="M_out", markershape=:star)
Plots.scatter!(data[1,nnpool.pools.==:Lin], data[2,nnpool.pools.==:Lin], color=:blue, label=:Lin, markershape = :utriangle)
p = Plots.scatter!(data[1,nnpool.pools.==:Lout], data[2,nnpool.pools.==:Lout], color=:red, label=:Lout, legend=:topleft)

In [ ]:
savefig(p, "plots/sample_masked.pdf")

In [ ]:
function neg_alignment(gamma)
    K_empiric = kernelmatrix(Val(:col), GaussianKernel(gamma), data)
    -alignment(centerkernelmatrix(K_empiric)[mask], centerkernelmatrix(K_opt)[mask])
end

pos_alignment(gamma) = -neg_alignment(gamma)

In [ ]:
plot(neg_alignment, 0, 10)

In [ ]:
res_pos = Optim.optimize(pos_alignment, 1.0, 100.0, abs_tol=0.1)
@show res_pos.minimizer

res = Optim.optimize(neg_alignment, 0.0, res_pos.minimizer, abs_tol=0.01)
@show res.minimizer
gamma_local = res.minimizer

##### Apply to SVDD

In [ ]:
model_1 = VanillaSVDD(data)
init_strategy = FixedParameterInitialization(GaussianKernel(gamma_local), 0.04)
initialize!(model_1, init_strategy)
fit!(model_1, solver)

p = Plots.plot(model_1, fill(:inlier, size(data,2)), color=:blues, legend=false, title="")

In [ ]:
savefig(p, "plots/sample_masked_svdd.pdf")